**Alumno: Isaias Eleuterio Tenorio Retis**

**TP2**

**Objetivo: Implementar un detector de máximo enfoque sobre un video aplicando técnicas de análisis espectral similar al que utilizan las cámaras digitales modernas. El video a procesar será “focus_video.mov”.**

**1) Se debe implementar un algoritmo que dada una imagen, o región, calcule la métrica propuesta en el paper r "Image
Sharpness Measure for Blurred Images in Frequency Domain" y realizar tres experimentos**

**Para cada experimento se debe presentar**
* Una curva o varias curvas que muestren la evolución de la métrica frame a frame donde se vea claramente cuando el algoritmo detecto el punto de máximo enfoque.
* Video con la ROI o matriz, graficada en rojo y superpuesta al video original para los frames que no están en foco y verde para los frames donde se detecta la condición de máximo enfoque.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
# Defino la métrica propuesta en el paper
def calcular_fm(image):
    # Paso 1: Calculamos F que es la representación de la Transformada de Fourier de la imagen
    F = np.fft.fft2(image)
    # Paso 2: Buscamos Fc, que se obtiene desplazando el origen de F al centro
    Fc = np.fft.fftshift(F)
    # Paso 3: Calcular AF que es el valor absoluto de la transformada de fourier centrada de la imagen
    AF = np.abs(Fc)
    # Paso 4: Calcular M  que es el valor máximo del componente de frecuencia en F
    M = np.max(AF)
    # Paso 5: Calculamos el umbral
    thres = M / 1000
    # Paso 6: Por último calculamos la métrica FM
    TH = np.sum(AF > thres)
    M, N = image.shape
    FM = TH / (M * N)

    return FM, thres

In [ ]:
# Cargamos el  video
cap = cv2.VideoCapture('focus_video.mov')

In [ ]:
# Defino el ROI o matriz
roi = None

**<font color="blue">Primer Experimento :</font> Medición sobre todo el frame**

In [ ]:
# Medición sobre todo el frame
fm_values = []
thres_values = []
colors = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    # Convertimos los frames a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Calculamos la métrica del paper sobre todo el frame
    fm, thres = calcular_fm(gray)
    print(f"FM: {fm:.4f}")
    fm_values.append(fm)
    thres_values.append(thres)
    # Ajustamos el tamaño del frame
    frame_small = cv2.resize(frame, (0, 0), fx=0.05, fy=0.05)
    # Superponemos un rectángulo en todo el frame
    if fm > 0.025:
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 255, 0), 20)  # Color verde
        colors.append('green')
    else:
        cv2.rectangle(frame, (0, 0), (frame.shape[1], frame.shape[0]), (0, 0, 255), 20)  # color rojo
        colors.append('red')
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Comando para encontrar el índice del valor máximo de FM
max_fm_index = fm_values.index(max(fm_values))

# Ahora graficamos una curva que muestra la evolución de la métrica FM
plt.scatter(range(len(fm_values)), fm_values, c=colors)
plt.xlabel('Frame')
plt.ylabel('Métrica FM')
plt.title('Evolución de la métrica FM')
plt.show()

# Por último imprimo el índice del frame con el máximo valor de FM
print(f"El frame con el máximo valor de FM es: {max_fm_index}")
print(f"El valor máximo de FM es: {max(fm_values)}")

**Observación:**
* <font color="blue">El frame con el máximo valor de FM es: 109.<font>
* <font color="blue">El valor máximo de FM es: 0.028624131944444443.</font>


**Conclusión:**
* <font color="blue">Los puntos verdes en el gráfico indican que la métrica FM es mayor que 0.025, lo que sugiere  que la imagen está relativamente clara o enfocada.</font>
* <font color="blue">Los puntos rojos indican que la métrica FM es menor o igual que 0.025, lo que sugiere que la imagen está borrosa o desenfocada.</font>

**<font color="blue">Segundo Experimento :</font> Medición sobre una ROI ubicada en el centro del frame. Área de la ROI = 5 o 10 % del área total del frame.**

In [ ]:
# Medición sobre una ROI ubicada en el centro del frame
# Cargamos el  video
cap = cv2.VideoCapture('focus_video.mov')
# 10% del área total del frame
roi_size = 0.10
fm_values = []
thres_values = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_small = cv2.resize(frame, (0, 0), fx=0.05, fy=0.05)  # Ajustamos el tamaño del frame
    # Convertimos el frame de color BGR a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Capturamos la altura y el ancho de las dimensiones del frame
    h, w = gray.shape
    # Calculamos el tamaño del ROI
    roi_h = int(h * roi_size)
    roi_w = int(w * roi_size)
    # Extraemos el ROI de la imagen en escala de grises
    roi = gray[h//2-roi_h//2:h//2+roi_h//2, w//2-roi_w//2:w//2+roi_w//2]
    # Calculamos la métrica y el umbral para el ROI
    fm, thres = calcular_fm(roi)
    print(f"FM: {fm:.4f}")  # Imprimo el valor del FM en cada frame
    fm_values.append(fm)
    thres_values.append(thres)

    # Superponer ROI en rojo o verde
    if fm > 0.6:
        color = (0, 255, 0)  # Verde
    else:
        color = (0, 0, 255)  # Rojo
    cv2.rectangle(frame, (w//2-roi_w//2, h//2-roi_h//2), (w//2+roi_w//2, h//2+roi_h//2), color, 2)

    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

# Comando para encontrar el índice del valor máximo de FM
max_fm_index = fm_values.index(max(fm_values))

# Ahora graficamos la curva de métrica FM
colors = ['green' if fm > 0.6 else 'red' for fm in fm_values]
plt.scatter(range(len(fm_values)), fm_values, c=colors)
plt.xlabel('Frame')
plt.ylabel('Métrica FM')
plt.title('Evolución de la métrica FM')
plt.show()

# Por último imprimo el índice del frame con el máximo valor de FM
print(f"El frame con el máximo valor de FM es: {max_fm_index}")
print(f"El valor máximo de FM es: {max(fm_values)}")

**Observación:**
*  <font color="blue">El frame con el máximo valor de FM es: 91.</font>
*  <font color="blue">El valor máximo de FM es: 0.7387152777777778.</font>


**Análisis:**
* <font color="blue">Los puntos verdes en el gráfico indican que la métrica FM es mayor que 0.6, lo que sugiere  que la imagen está relativamente clara o enfocada.</font>
* <font color="blue">Los puntos rojos indican que la métrica FM es menor o igual que 0.6, lo que sugiere que la imagen está borrosa o desenfocada.</font>


**<font color="blue">Tercer Experimento : </font> Medición sobre una matriz de enfoque compuesta por un arreglo de NxM elementos rectangulares equiespaciados. N y M son valores arbitrarios, probar con varios valores 3x3,7x5,etc ... (al menos 3)**

* <font color="blue">Área de la ROI a considerar es del 10%.</font>
* <font color="blue"> FM a considerar para todas las matrices es del 0.5.</font>

**Primero: Matriz de orden 3x3**

In [ ]:
# Defino la matriz de enfoque
N = 3  # Número de filas
M = 3  # Número de columnas
roi_size = 0.1  # tamaño del ROI

# Cargamos el video
cap = cv2.VideoCapture('focus_video.mov')

# Listas para almacenar los valores de FM y umbrales
fm_values = []
thres_values = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertimos el frame de color BGR a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Capturamos la altura y el ancho de las dimensiones del frame
    h, w = gray.shape

    # Calculamos el centro del frame
    center_x, center_y = w // 2, h // 2

    # Matriz para almacenar los valores de FM para cada elemento de la matriz de enfoque
    fm_matrix = np.zeros((N, M))

    # Calcular el tamaño del ROI en píxeles
    roi_w = int(w * roi_size)
    roi_h = int(h * roi_size)

    # Ajustamos para que los rectángulos se dibujen alrededor del centro
    start_x = center_x - (M // 2) * roi_w
    start_y = center_y - (N // 2) * roi_h

    # Iterar sobre cada elemento de la matriz de enfoque
    for i in range(N):
        for j in range(M):
            # Calcular las coordenadas del ROI centrado
            roi_x = start_x + j * roi_w
            roi_y = start_y + i * roi_h
            roi = gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

            # Calculamos la métrica FM para el ROI actual
            fm, _ = calcular_fm(roi)

            # Asignamos el valor de FM a la matriz de enfoque
            fm_matrix[i, j] = fm

    # Almacenamos los valores de FM y umbrales para cada frame
    fm_values.append(fm_matrix.flatten())
    thres_values.append(np.zeros_like(fm_matrix.flatten()))

    # Superponer ROI en rojo o verde
    for i in range(N):
        for j in range(M):
            if fm_matrix[i, j] > 0.5:
                color = (0, 255, 0)  # Verde
            else:
                color = (0, 0, 255)  # Rojo
            cv2.rectangle(frame,
              (start_x + j * roi_w, start_y + i * roi_h),
              (start_x + j * roi_w + roi_w, start_y + i * roi_h + roi_h),
              color, 2)

    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Graficamos la curva de métrica FM para cada elemento de la matriz de enfoque
fig, axs = plt.subplots(N, M, figsize=(15, 10), sharex=True, sharey=True)
for i in range(N):
    for j in range(M):
        colors = ['green' if fm > 0.5 else 'red' for fm in np.array(fm_values)[:, i*M + j]]
        axs[i, j].scatter(range(len(np.array(fm_values)[:, i*M + j])), np.array(fm_values)[:, i*M + j], c=colors, s=10)
        axs[i, j].set_title(f'({i}, {j})')

fig.tight_layout()
plt.show()

# Comando para encontrar el índice del valor máximo de FM
indice_max_fm = np.array(fm_values).argmax()

# Por último imprimimos el índice del frame con el máximo valor de FM
print(f"El frame con el máximo valor de FM es: {indice_max_fm}")
print(f"El valor máximo de FM es: {np.array(fm_values).max()}")

**Conclusión:**

<font color="blue">
El análisis de la métrica FM reveló variabilidad en su valor máximo entre frames, sugiriendo que captura diferentes aspectos de la imagen en distintos momentos. Esta variabilidad puede deberse a cambios en la escena o sensibilidad a características de la imagen. Aunque la métrica FM es útil para evaluar la calidad de enfoque, es crucial considerar su variabilidad para obtener resultados precisos
</font>

**Segundo: Matriz de orden 7x5**

In [ ]:
# Defino la matriz de enfoque
N = 7  # Número de filas
M = 5  # Número de columnas
roi_size = 0.1  # tamaño del ROI

# Cargamos el video
cap = cv2.VideoCapture('focus_video.mov')

# Listas para almacenar los valores de FM y umbrales
fm_values = []
thres_values = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertimos el frame de color BGR a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Capturamos la altura y el ancho de las dimensiones del frame
    h, w = gray.shape

    # Calculamos el centro del frame
    center_x, center_y = w // 2, h // 2

    # Matriz para almacenar los valores de FM para cada elemento de la matriz de enfoque
    fm_matrix = np.zeros((N, M))

    # Calcular el tamaño del ROI en píxeles
    roi_w = int(w * roi_size)
    roi_h = int(h * roi_size)

    # Ajustamos para que los rectángulos se dibujen alrededor del centro
    start_x = center_x - (M // 2) * roi_w
    start_y = center_y - (N // 2) * roi_h

    # Iterar sobre cada elemento de la matriz de enfoque
    for i in range(N):
        for j in range(M):
            # Calcular las coordenadas del ROI centrado
            roi_x = start_x + j * roi_w
            roi_y = start_y + i * roi_h
            roi = gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

            # Calculamos la métrica FM para el ROI actual
            fm, _ = calcular_fm(roi)

            # Asignamos el valor de FM a la matriz de enfoque
            fm_matrix[i, j] = fm

    # Almacenamos los valores de FM y umbrales para cada frame
    fm_values.append(fm_matrix.flatten())
    thres_values.append(np.zeros_like(fm_matrix.flatten()))

    # Superponer ROI en rojo o verde
    for i in range(N):
        for j in range(M):
            if fm_matrix[i, j] > 0.5:
                color = (0, 255, 0)  # Verde
            else:
                color = (0, 0, 255)  # Rojo
            cv2.rectangle(frame,
              (start_x + j * roi_w, start_y + i * roi_h),
              (start_x + j * roi_w + roi_w, start_y + i * roi_h + roi_h),
              color, 2)

    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Graficamos la curva de métrica FM para cada elemento de la matriz de enfoque
fig, axs = plt.subplots(N, M, figsize=(15, 10), sharex=True, sharey=True)
for i in range(N):
    for j in range(M):
        colors = ['green' if fm > 0.5 else 'red' for fm in np.array(fm_values)[:, i*M + j]]
        axs[i, j].scatter(range(len(np.array(fm_values)[:, i*M + j])), np.array(fm_values)[:, i*M + j], c=colors, s=10)
        axs[i, j].set_title(f'({i}, {j})')

fig.tight_layout()
plt.show()

**Conclusión:**

<font color="blue">La matriz de 7x5 utilizada para analizar la métrica FM mostró una distribución dispersa de los valores de FM mayores a 0.5. Esto indica que la calidad de enfoque varía en diferentes regiones de la imagen, y que la métrica FM es capaz de capturar estas variaciones.</font>

**Tercero: Matriz de orden 8x6**

In [ ]:
# Defino la matriz de enfoque
N = 8  # Número de filas
M = 6  # Número de columnas
roi_size = 0.1  # tamaño del ROI

# Cargamos el video
cap = cv2.VideoCapture('focus_video.mov')

# Listas para almacenar los valores de FM y umbrales
fm_values = []
thres_values = []

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convertimos el frame de color BGR a escala de grises
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Capturamos la altura y el ancho de las dimensiones del frame
    h, w = gray.shape

    # Calculamos el centro del frame
    center_x, center_y = w // 2, h // 2

    # Matriz para almacenar los valores de FM para cada elemento de la matriz de enfoque
    fm_matrix = np.zeros((N, M))

    # Calcular el tamaño del ROI en píxeles
    roi_w = int(w * roi_size)
    roi_h = int(h * roi_size)

    # Ajustamos para que los rectángulos se dibujen alrededor del centro
    start_x = center_x - (M // 2) * roi_w
    start_y = center_y - (N // 2) * roi_h

    # Iterar sobre cada elemento de la matriz de enfoque
    for i in range(N):
        for j in range(M):
            # Calcular las coordenadas del ROI centrado
            roi_x = start_x + j * roi_w
            roi_y = start_y + i * roi_h
            roi = gray[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]

            # Calculamos la métrica FM para el ROI actual
            fm, _ = calcular_fm(roi)

            # Asignamos el valor de FM a la matriz de enfoque
            fm_matrix[i, j] = fm

    # Almacenamos los valores de FM y umbrales para cada frame
    fm_values.append(fm_matrix.flatten())
    thres_values.append(np.zeros_like(fm_matrix.flatten()))

    # Superponer ROI en rojo o verde
    for i in range(N):
        for j in range(M):
            if fm_matrix[i, j] > 0.5:
                color = (0, 255, 0)  # Verde
            else:
                color = (0, 0, 255)  # Rojo
            cv2.rectangle(frame,
              (start_x + j * roi_w, start_y + i * roi_h),
              (start_x + j * roi_w + roi_w, start_y + i * roi_h + roi_h),
              color, 2)

    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Graficamos la curva de métrica FM para cada elemento de la matriz de enfoque
fig, axs = plt.subplots(N, M, figsize=(15, 10), sharex=True, sharey=True)
for i in range(N):
    for j in range(M):
        colors = ['green' if fm > 0.5 else 'red' for fm in np.array(fm_values)[:, i*M + j]]
        axs[i, j].scatter(range(len(np.array(fm_values)[:, i*M + j])), np.array(fm_values)[:, i*M + j], c=colors, s=10)
        axs[i, j].set_title(f'({i}, {j})')

fig.tight_layout()
plt.show()

**Observación:**

<font color="blue">La matriz de 8x6 utilizada para analizar la métrica FM demostró que la capacidad de la métrica para capturar variaciones en la calidad de enfoque se mantiene incluso en matrices de mayor tamaño y complejidad, lo que sugiere una robustez y fiabilidad en la métrica.</font>

**Conclusión Final**

<font color="blue">En conclusión, la métrica FM es una herramienta efectiva para analizar la calidad de enfoque en imágenes, mostrando robustez y fiabilidad en diferentes contextos. Su variabilidad refleja la complejidad de la imagen, lo que la hace útil para evaluar la calidad de enfoque de manera precisa.</font>